<p style="font-size:20px">Prezentacja - Keras z użyciem sieci konwolucyjnych, Mateusz Rogowski</p>

# Keras

![BG9G6VAKJKGVTEUGDORQM1XBEK4DBOO1s.png](files/attachments/BG9G6VAKJKGVTEUGDORQM1XBEK4DBOO1s.png)

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn import metrics

In [2]:
data_path = '/data/MNIST/'
train_csv = os.path.join(data_path, 'train.csv')
test_csv = os.path.join(data_path, 'test.csv')

In [3]:
train = pd.read_csv(train_csv)
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.shape

(42000, 785)

In [5]:
test = pd.read_csv(test_csv)
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test.shape

(28000, 784)

# Prepare data

In [7]:
from sklearn import preprocessing

In [8]:
X = train.loc[:,'pixel0':].values
# y_binarizer = preprocessing.LabelBinarizer()
y = train['label'].values
y_binarized = preprocessing.label_binarize(y, np.arange(10))

In [9]:
X = X / 255

In [10]:
from sklearn import model_selection
sss = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=.3, random_state=0)
train_idx, test_idx = list(sss.split(X, y))[0]

X_train, y_train, y_binarized_train = X[train_idx], y[train_idx], y_binarized[train_idx]
X_test, y_test, y_binarized_test = X[test_idx], y[test_idx], y_binarized[test_idx]

# Try convolutional network

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, GaussianDropout

Using TensorFlow backend.


In [14]:
from keras import callbacks
from keras.utils.generic_utils import Progbar

class CustomProgbarLogger(callbacks.ProgbarLogger):
    '''Callback that prints metrics to stdout.
    '''
    def __init__(self, count_mode='samples', step_size=1):
        self.step_size=step_size
        super().__init__(count_mode)
    
    def on_train_begin(self, logs={}):
        self.verbose = self.verbose_bak = self.params['verbose'] or 1
        self.epochs = self.params['epochs']

    def on_epoch_begin(self, epoch, logs={}):
        self.verbose = self.verbose_bak & (((epoch % self.step_size) == 0) | (epoch == (self.epochs-1)))
        super().on_epoch_begin(epoch, logs)

In [26]:
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Flatten, Dropout, Dense, noise
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

## Standard generator

In [25]:
X_2d_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_2d_test = X_test.reshape(X_test.shape[0], 1, 28, 28)

In [27]:
img_width, img_height = 28, 28
output_size = y_binarized.shape[1]

def build_model_conv():
    model = Sequential()
    
    # build the rest of the network
    model.add(ZeroPadding2D((1, 1), input_shape=(1, img_width, img_height)))
    model.add(Conv2D(28, (5, 5), activation='relu', name='conv1_1', padding='same'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(28, (3, 3), activation='relu', name='conv1_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(noise.GaussianNoise(0.1))
    model.add(Dense(128, activation='relu', kernel_constraint=maxnorm(3)))
    
    model.add(Dropout(0.35))
    model.add(GaussianDropout(0.25))
    model.add(Dense(64, activation='tanh'))
    
    model.add(Dropout(0.4))
    model.add(Dense(output_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [28]:
batch_size = 4096
epochs = 20
nb_val_samples = X_test.shape[0] - X_test.shape[0]%batch_size
model = build_model_conv()

In [29]:
epochs = 100

history = model.fit(X_2d_train, y_binarized_train,
                    epochs=epochs, 
                    validation_data=(X_2d_test, y_binarized_test),
                    batch_size=batch_size,
                    verbose=0,
                    callbacks=[CustomProgbarLogger(step_size=5)])

Epoch 1/20
29400/29400 [==============================] - 1s - loss: 2.2488 - acc: 0.1698 - val_loss: 1.9689 - val_acc: 0.5936
Epoch 6/20
29400/29400 [==============================] - 0s - loss: 0.5642 - acc: 0.8293 - val_loss: 0.3273 - val_acc: 0.9055
Epoch 11/20
29400/29400 [==============================] - 0s - loss: 0.3069 - acc: 0.9125 - val_loss: 0.1961 - val_acc: 0.9399
Epoch 16/20
29400/29400 [==============================] - 0s - loss: 0.2184 - acc: 0.9391 - val_loss: 0.1433 - val_acc: 0.9572
Epoch 20/20
29400/29400 [==============================] - 0s - loss: 0.1755 - acc: 0.9520 - val_loss: 0.1184 - val_acc: 0.9656


## Fit with image generator

In [30]:
img_width, img_height = 28, 28
output_size = y_binarized.shape[1]

def build_model_conv():
    model = Sequential()
    
    # build the rest of the network
    model.add(ZeroPadding2D((1, 1), input_shape=(1, img_width, img_height)))
    model.add(Conv2D(28, (5, 5), activation='relu', name='conv1_1', padding='same'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(28, (3, 3), activation='relu', name='conv1_2'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='conv2_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(noise.GaussianNoise(0.1))
    model.add(Dense(128, activation='relu', kernel_constraint=maxnorm(3)))
    
    model.add(Activation('tanh'))
    model.add(Dropout(0.35))
    model.add(Dense(128))
    
    model.add(Dropout(0.35))
    model.add(GaussianDropout(0.25))
    model.add(Dense(64, activation='tanh'))
    
    model.add(Dropout(0.4))
    model.add(Dense(output_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [37]:
batch_size = 1024 * 8
epochs = 2000
nb_val_samples = X_test.shape[0] - X_test.shape[0]%batch_size
model = build_model_conv()

In [38]:
datagen = ImageDataGenerator(
    data_format="channels_first",
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_2d_train)

In [39]:
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(generator=datagen.flow(X_2d_train, y_binarized_train, batch_size=batch_size),
                              epochs=epochs, 
                              steps_per_epoch=X_2d_train.shape[0] // batch_size,
                              validation_data=(X_2d_test, y_binarized_test),
                              verbose=0,
                              callbacks=[CustomProgbarLogger(count_mode="steps", step_size=5)])

Epoch 1/2000
3/3 [==============================] - 2s - loss: 2.3413 - acc: 0.1030 - val_loss: 2.2924 - val_acc: 0.1067
Epoch 6/2000
3/3 [==============================] - 2s - loss: 2.0972 - acc: 0.2584 - val_loss: 1.7385 - val_acc: 0.5327
Epoch 11/2000
3/3 [==============================] - 2s - loss: 1.6540 - acc: 0.3987 - val_loss: 1.2403 - val_acc: 0.6148
Epoch 16/2000
3/3 [==============================] - 1s - loss: 1.3705 - acc: 0.4990 - val_loss: 0.9438 - val_acc: 0.7153
Epoch 21/2000
3/3 [==============================] - 2s - loss: 1.1439 - acc: 0.5945 - val_loss: 0.6794 - val_acc: 0.7889
Epoch 26/2000
3/3 [==============================] - 2s - loss: 0.9316 - acc: 0.6825 - val_loss: 0.4996 - val_acc: 0.8342
Epoch 31/2000
3/3 [==============================] - 2s - loss: 0.7978 - acc: 0.7284 - val_loss: 0.4027 - val_acc: 0.8563
Epoch 36/2000
3/3 [==============================] - 1s - loss: 0.7064 - acc: 0.7609 - val_loss: 0.3561 - val_acc: 0.8635
Epoch 41/2000
3/3 [=======

3/3 [==============================] - 2s - loss: 0.0828 - acc: 0.9776 - val_loss: 0.0375 - val_acc: 0.9911
Epoch 666/2000
3/3 [==============================] - 2s - loss: 0.0832 - acc: 0.9784 - val_loss: 0.0353 - val_acc: 0.9919
Epoch 671/2000
3/3 [==============================] - 2s - loss: 0.0848 - acc: 0.9777 - val_loss: 0.0316 - val_acc: 0.9929
Epoch 676/2000
3/3 [==============================] - 1s - loss: 0.0911 - acc: 0.9762 - val_loss: 0.0333 - val_acc: 0.9922
Epoch 681/2000
3/3 [==============================] - 2s - loss: 0.0855 - acc: 0.9768 - val_loss: 0.0324 - val_acc: 0.9923
Epoch 686/2000
3/3 [==============================] - 2s - loss: 0.0872 - acc: 0.9770 - val_loss: 0.0377 - val_acc: 0.9906
Epoch 691/2000
3/3 [==============================] - 2s - loss: 0.0784 - acc: 0.9789 - val_loss: 0.0328 - val_acc: 0.9921
Epoch 696/2000
3/3 [==============================] - 1s - loss: 0.0877 - acc: 0.9751 - val_loss: 0.0372 - val_acc: 0.9915
Epoch 701/2000
3/3 [===========

3/3 [==============================] - 2s - loss: 0.0545 - acc: 0.9854 - val_loss: 0.0303 - val_acc: 0.9935
Epoch 1326/2000
3/3 [==============================] - 2s - loss: 0.0564 - acc: 0.9841 - val_loss: 0.0291 - val_acc: 0.9933
Epoch 1331/2000
3/3 [==============================] - 2s - loss: 0.0492 - acc: 0.9867 - val_loss: 0.0279 - val_acc: 0.9941
Epoch 1336/2000
3/3 [==============================] - 1s - loss: 0.0539 - acc: 0.9861 - val_loss: 0.0310 - val_acc: 0.9935
Epoch 1341/2000
3/3 [==============================] - 2s - loss: 0.0524 - acc: 0.9859 - val_loss: 0.0320 - val_acc: 0.9930
Epoch 1346/2000
3/3 [==============================] - 2s - loss: 0.0567 - acc: 0.9857 - val_loss: 0.0260 - val_acc: 0.9939
Epoch 1351/2000
3/3 [==============================] - 2s - loss: 0.0595 - acc: 0.9848 - val_loss: 0.0294 - val_acc: 0.9936
Epoch 1356/2000
3/3 [==============================] - 2s - loss: 0.0562 - acc: 0.9849 - val_loss: 0.0284 - val_acc: 0.9937
Epoch 1361/2000
3/3 [===

3/3 [==============================] - 2s - loss: 0.0404 - acc: 0.9895 - val_loss: 0.0265 - val_acc: 0.9944
Epoch 1986/2000
3/3 [==============================] - 2s - loss: 0.0426 - acc: 0.9894 - val_loss: 0.0261 - val_acc: 0.9943
Epoch 1991/2000
3/3 [==============================] - 2s - loss: 0.0395 - acc: 0.9901 - val_loss: 0.0270 - val_acc: 0.9939
Epoch 1996/2000
3/3 [==============================] - 1s - loss: 0.0444 - acc: 0.9882 - val_loss: 0.0266 - val_acc: 0.9946
Epoch 2000/2000
3/3 [==============================] - 1s - loss: 0.0408 - acc: 0.9900 - val_loss: 0.0277 - val_acc: 0.9945
